<a href="https://colab.research.google.com/github/Ananduak7/A-HYBRID-MULTI-MODEL-BASED-APPROACH-FOR-HEART-DISEASE-DETECTION-AND-CLASSIFICATION/blob/main/FINAL_INTEGRATIONipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print(os.path.exists("/content/e00055.wav"))


True


Risk Prioritization Logic:
Audio Model (Heart Sound Analysis)

Unhealthy (1.0 probability) → High risk
Healthy (1.0 probability) → Low risk
Borderline cases (0.4 - 0.6 probability) → Medium risk
ECG Model (Image Classification)

Myocardial Infarction & History of Myocardial Infarction → Highest risk
Abnormal Heartbeat → Medium risk
Normal → Low risk
Both results are combined to generate a final risk score (0-100%) and risk category (Low/Medium/High/Severe).

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# ✅ Load Audio Classification Model (BiLSTM)
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
audio_model = tf.keras.models.load_model(AUDIO_MODEL_PATH)

# ✅ Load ECG Classification Model (EfficientNet + ResNet + MLP)
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"

ecg_model = load_model(ECG_MODEL_PATH)

# ✅ Load Label Encoder for ECG Class Labels
with open(ENCODER_PATH, "rb") as f:
    label_encoder = pickle.load(f)

# ✅ Load Pre-trained Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Define Class Labels for ECG Model
print("Available ECG Class Labels:", label_encoder.classes_)

ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}

AUDIO_RISK_WEIGHTS = {
    "Healthy": 0.1,
    "Unhealthy": 0.9
}

# ✅ Function to Preprocess Audio File
def preprocess_audio(file_path, max_pad_len=400):
    try:
        audio, sample_rate = librosa.load(file_path, sr=2000)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

        # Padding or truncating to ensure consistent input length
        pad_width = max_pad_len - mfccs.shape[1]
        if pad_width > 0:
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_pad_len]

        mfccs = mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)
        return mfccs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# ✅ Function to Predict Heart Condition from Audio
def detect_heart_disease(file_path):
    features = preprocess_audio(file_path)
    if features is None:
        print("Error in preprocessing the audio file.")
        return None, 0.0

    prediction = audio_model.predict(features)
    confidence = float(prediction[0])  # Probability of being Unhealthy
    predicted_class = "Unhealthy" if confidence > 0.5 else "Healthy"
    confidence = confidence if predicted_class == "Unhealthy" else 1 - confidence  # Adjust confidence

    print(f"🔊 Audio Prediction: {predicted_class} (Confidence: {confidence:.2f})")
    return predicted_class, confidence

# ✅ Function to Preprocess ECG Image
def preprocess_image(image_path):
    if not os.path.exists(image_path):
        raise ValueError(f"Error: File does not exist at {image_path}")

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Error: Unable to read the image. Check if the file format is supported.")

    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# ✅ Function to Predict Heart Condition from ECG Image
def predict_disease(image_path):
    img = preprocess_image(image_path)

    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)

    final_features = np.hstack((eff_features, res_features))

    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)
    predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    confidence = float(np.max(prediction))

    print(f"📟 ECG Prediction: {predicted_class} (Confidence: {confidence:.2f})")
    return predicted_class, confidence

# ✅ Function to Compute Risk Score
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence):
    if ecg_result not in ECG_RISK_WEIGHTS:
        raise KeyError(f"Unknown ECG class detected: {ecg_result}. Please check class labels.")

    # Compute risk values
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS[ecg_result] * ecg_confidence

    final_risk_score = (audio_risk + ecg_risk) / 2  # Weighted average
    print(f"🩺 Patient's Risk Score: {final_risk_score:.2f} (0 - Low, 1 - High)")

    # Interpret risk level
    if final_risk_score >= 0.8:
        print("⚠️ High Risk! Immediate medical attention recommended.")
    elif final_risk_score >= 0.5:
        print("⚠️ Moderate Risk. Consult a cardiologist soon.")
    else:
        print("✅ Low Risk. Maintain a healthy lifestyle.")

    return final_risk_score

# ✅ Main Function to Evaluate Patient's Health
def evaluate_patient(audio_file_path, ecg_file_path):
    print("\n🔍 Evaluating Patient's Heart Health...\n")

    audio_result, audio_confidence = detect_heart_disease(audio_file_path)
    ecg_result, ecg_confidence = predict_disease(ecg_file_path)

    if audio_result is None or ecg_result is None:
        print("❌ Error in processing patient data.")
        return

    risk_score = calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence)
    return risk_score

# ✅ Example Usage
AUDIO_FILE_PATH = "/content/f0108.wav"
ECG_IMAGE_PATH = "/content/HB(3).jpg"

evaluate_patient(AUDIO_FILE_PATH, ECG_IMAGE_PATH)


Available ECG Class Labels: ['ECG Images of Myocardial Infarction Patients (240x12=2880)'
 'ECG Images of Patient that have History of MI (172x12=2064)'
 'ECG Images of Patient that have abnormal heartbeat (233x12=2796)'
 'Normal Person ECG Images (284x12=3408)']

🔍 Evaluating Patient's Heart Health...



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step


<ipython-input-4-08497c3a9641>:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence = float(prediction[0])  # Probability of being Unhealthy


🔊 Audio Prediction: Healthy (Confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
📟 ECG Prediction: ECG Images of Patient that have abnormal heartbeat (233x12=2796) (Confidence: 0.94)
🩺 Patient's Risk Score: 0.33 (0 - Low, 1 - High)
✅ Low Risk. Maintain a healthy lifestyle.


0.3322629512133972

In [ ]:
# Print available class names
print("Available ECG Class Labels:", label_encoder.classes_)

# Define a normalized risk weight dictionary


Available ECG Class Labels: ['ECG Images of Myocardial Infarction Patients (240x12=2880)'
 'ECG Images of Patient that have History of MI (172x12=2064)'
 'ECG Images of Patient that have abnormal heartbeat (233x12=2796)'
 'Normal Person ECG Images (284x12=3408)']


In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import logging
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# ✅ Set Up Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ✅ Load Audio Classification Model (BiLSTM)
try:
    AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
    audio_model = tf.keras.models.load_model(AUDIO_MODEL_PATH)
    logging.info("Audio classification model loaded successfully.")
except Exception as e:
    logging.error(f"Failed to load audio model: {e}")

# ✅ Load ECG Classification Model (EfficientNet + ResNet + MLP)
try:
    ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
    ENCODER_PATH = "/content/label_encoder.pkl"
    ecg_model = load_model(ECG_MODEL_PATH)
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
    logging.info("ECG classification model and label encoder loaded successfully.")
except Exception as e:
    logging.error(f"Failed to load ECG model or label encoder: {e}")

# ✅ Load Pre-trained Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Define Class Labels for ECG Model
try:
    print("Available ECG Class Labels:", label_encoder.classes_)
except Exception as e:
    logging.error(f"Failed to read label encoder classes: {e}")

# ✅ Define Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}

AUDIO_RISK_WEIGHTS = {
    "Healthy": 0.1,
    "Unhealthy": 0.9
}

# ✅ Function to Preprocess Audio File
def preprocess_audio(file_path, max_pad_len=400):
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Audio file not found: {file_path}")

        audio, sample_rate = librosa.load(file_path, sr=2000)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

        # Padding or truncating to ensure consistent input length
        pad_width = max_pad_len - mfccs.shape[1]
        if pad_width > 0:
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_pad_len]

        mfccs = mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)
        return mfccs
    except Exception as e:
        logging.error(f"Error processing {file_path}: {e}")
        return None

# ✅ Function to Predict Heart Condition from Audio
def detect_heart_disease(file_path):
    features = preprocess_audio(file_path)
    if features is None:
        logging.error("Error in preprocessing the audio file.")
        return None, 0.0

    prediction = audio_model.predict(features)
    confidence = float(prediction[0])  # Probability of being Unhealthy
    predicted_class = "Unhealthy" if confidence > 0.5 else "Healthy"
    confidence = confidence if predicted_class == "Unhealthy" else 1 - confidence  # Adjust confidence

    logging.info(f"🔊 Audio Prediction: {predicted_class} (Confidence: {confidence:.2f})")
    return predicted_class, confidence

# ✅ Function to Preprocess ECG Image
def preprocess_image(image_path):
    try:
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"ECG file not found: {image_path}")

        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("Unable to read the ECG image.")

        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        return img
    except Exception as e:
        logging.error(f"Error processing ECG image: {e}")
        return None

# ✅ Function to Predict Heart Condition from ECG Image
def predict_disease(image_path):
    img = preprocess_image(image_path)
    if img is None:
        return None, 0.0

    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))

    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)
    predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    confidence = float(np.max(prediction))

    logging.info(f"📟 ECG Prediction: {predicted_class} (Confidence: {confidence:.2f})")
    return predicted_class, confidence

# ✅ Function to Compute Risk Score
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence):
    if ecg_result not in ECG_RISK_WEIGHTS:
        logging.error(f"Unknown ECG class detected: {ecg_result}.")
        return None

    # Compute risk values
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS[ecg_result] * ecg_confidence

    final_risk_score = (audio_risk + ecg_risk) / 2  # Weighted average
    logging.info(f"🩺 Patient's Risk Score: {final_risk_score:.2f} (0 - Low, 1 - High)")

    return final_risk_score

# ✅ Main Function to Evaluate Patient's Health
def evaluate_patient(audio_file_path, ecg_file_path):
    logging.info("\n🔍 Evaluating Patient's Heart Health...\n")

    audio_result, audio_confidence = detect_heart_disease(audio_file_path)
    ecg_result, ecg_confidence = predict_disease(ecg_file_path)

    if audio_result is None or ecg_result is None:
        logging.error("❌ Error in processing patient data.")
        return

    risk_score = calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence)
    return risk_score

# ✅ Example Usage
AUDIO_FILE_PATH = "/content/f0108.wav"
ECG_IMAGE_PATH = "/content/HB(3).jpg"

evaluate_patient(AUDIO_FILE_PATH, ECG_IMAGE_PATH)


Available ECG Class Labels: ['ECG Images of Myocardial Infarction Patients (240x12=2880)'
 'ECG Images of Patient that have History of MI (172x12=2064)'
 'ECG Images of Patient that have abnormal heartbeat (233x12=2796)'
 'Normal Person ECG Images (284x12=3408)']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step


<ipython-input-5-13aa3e5e2c70>:87: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence = float(prediction[0])  # Probability of being Unhealthy


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


0.3322629512133972

In [ ]:
!pip install scikit-learn==1.4.1.post1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
import pandas as pd

df = pd.read_excel("/content/new.xlsx")
print(df.head())  # Check the first few rows





          0         1         2         3         4         5         6  \
0  0.050481 -0.339781  0.265111  0.045092  0.040086  0.087173  0.092814   
1 -0.019198 -0.012369  0.049249  0.042435  0.102515  0.086274 -0.179430   
2  0.022934  0.006696  0.041837  0.071592  0.068073  0.091548  0.114399   
3  0.017883  0.036710  0.004640  0.038210  0.085609  0.075911  0.116421   
4 -0.050635 -0.254571  0.004206  0.064948  0.113252  0.093907  0.094563   

          7         8         9  ...      1991      1992      1993      1994  \
0  0.099586 -0.143305  0.164186  ... -0.401426 -0.402333 -0.373426 -0.303710   
1  0.182514  0.144317  0.163252  ... -0.378149 -0.162815 -0.295464 -0.106774   
2  0.116452  0.178171  0.160357  ... -0.226982 -0.209063 -0.197806 -0.162262   
3  0.148803  0.164897  0.167658  ... -0.212147 -0.197705 -0.179078 -0.181514   
4  0.140718  0.160440  0.139388  ... -0.424251 -0.377917 -0.341359 -0.306715   

       1995      1996      1997      1998      1999   Label  
0  0.0

In [ ]:
!pip install --upgrade scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.1.post1
    Uninstalling scikit-learn-1.4.1.post1:
      Successfully uninstalled scikit-learn-1.4.1.post1


In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models (Suppressing Compilation Warning)
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/ppg_scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder Safely
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixing 'Label' KeyError)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove non-numeric column
    else:
        print("Warning: 'Label' column not found in PPG data. Proceeding without it.")
    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # Load PPG Data Safely
    try:
        df_ppg = pd.read_excel(ppg_file)  # Read .xlsx instead of .csv
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})  # Convert 'MI' to 1 and 'Normal' to 0
        else:
            print("Warning: 'Label' column not found in PPG data.")
        ppg_row = df_ppg.dropna().iloc[0]  # Take the first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/f0108.wav", "/content/HB(3).jpg", "/content/new_fixed_corrected.xlsx")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 747ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Audio Result: Healthy (Confidence: 1.00)
ECG Result: ECG Images of Patient that have abnormal heartbeat (233x12=2796) (Confidence: 0.94)
PPG Result: Myocardial Infarction (Confidence: 1.00)
Final Risk Score: 0.52


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-19-e76756006459>:124: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-19-e76756006459>:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.5200154614279883

The risk score is calculated using the formula:

Final Risk Score
=
Audio Risk
+
ECG Risk
+
PPG Risk
3
Final Risk Score=
3
Audio Risk+ECG Risk+PPG Risk
​

where:

Audio Risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
ECG Risk = ECG_RISK_WEIGHTS[ecg_result] * ecg_confidence
PPG Risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
Given:
ECG Result = Normal

ECG_RISK_WEIGHTS["Normal Person ECG Images (284x12=3408)"] = 0.1
ECG Confidence: Let's assume 0.9 (Example)
ECG Risk =
0.1
×
0.9
=
0.09
0.1×0.9=0.09
Heart Sound Result = Unhealthy

AUDIO_RISK_WEIGHTS["Unhealthy"] = 0.9
Audio Confidence: Let's assume 0.8 (Example)
Audio Risk =
0.9
×
0.8
=
0.72
0.9×0.8=0.72
PPG Result = Myocardial Infarction (MI)

PPG_RISK_WEIGHTS[1] = 0.9
PPG Confidence: Let's assume 0.85 (Example)
PPG Risk =
0.9
×
0.85
=
0.765
0.9×0.85=0.765
Final Risk Score Calculation:
Final Risk Score
=
0.72
+
0.09
+
0.765
3
Final Risk Score=
3
0.72+0.09+0.765
​

=
1.575
3
=
0.525
=
3
1.575
​
 =0.525
Final Risk Score = 0.53 (Rounded to Two Decimal Places)
This means the overall heart disease risk is moderate (52.5%), influenced mostly by the Unhealthy Heart Sound and PPG MI prediction, while the Normal ECG lowers the overall risk.






You said:
FROM WHERE DID YOU GET THOSE CONFIDENCE
ChatGPT said:
The confidence values for each prediction come from the model outputs. In your code, each model predicts a probability or confidence score. Let's break it down:

1. Audio Confidence
In the function detect_heart_disease(audio_path), the heart sound model predicts a probability:

python
Copy
Edit
prediction = audio_model.predict(features)[0]  # Returns probability between 0 and 1
predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
If the model predicts 0.8, and the result is Unhealthy, then:
Audio Confidence = 0.8
2. ECG Confidence
In predict_disease(ecg_path), the ECG model predicts probabilities for multiple classes. The highest probability is taken:

python
Copy
Edit
prediction = ecg_model.predict(final_features)  # Returns array of probabilities
predicted_class_index = np.argmax(prediction)  # Get index of max probability
predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
confidence = float(np.max(prediction))  # Take highest probability as confidence
If the Normal ECG class has the highest probability 0.9, then:
ECG Confidence = 0.9
3. PPG Confidence
In predict_ppg(ppg_row), the model gives a probability for Myocardial Infarction (MI):

python
Copy
Edit
prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]  # Probability of MI
predicted_class = 1 if prediction_prob > 0.5 else 0  # 1 = MI, 0 = Normal
If the model predicts 0.85 for MI, then:
PPG Confidence = 0.85
Summary of Confidence Values
Audio Confidence = 0.8 (from heart sound model)
ECG Confidence = 0.9 (from ECG model)
PPG Confidence = 0.85 (from PPG model)
These confidence scores are used in the risk score calculation:

Final Risk Score
=
(
0.9
×
0.8
)
+
(
0.1
×
0.9
)
+
(
0.9
×
0.85
)
3
Final Risk Score=
3
(0.9×0.8)+(0.1×0.9)+(0.9×0.85)
​

=
0.72
+
0.09
+
0.765
3
=
0.525
=
3
0.72+0.09+0.765
​
 =0.525
So, the risk score is 0.53 (or 53%), meaning a moderate risk of heart disease.

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models (Suppressing Compilation Warning)
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/ppg_scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder Safely
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixing 'Label' KeyError)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove non-numeric column
    else:
        print("Warning: 'Label' column not found in PPG data. Proceeding without it.")
    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # Load PPG Data Safely
    try:
        df_ppg = pd.read_excel(ppg_file)  # Read .xlsx instead of .csv
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})  # Convert 'MI' to 1 and 'Normal' to 0
        else:
            print("Warning: 'Label' column not found in PPG data.")
        ppg_row = df_ppg.dropna().iloc[0]  # Take the first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/UNHEALTHYSOUND.wav", "/content/Normal(3).jpg", "/content/new_fixed_corrected.xlsx")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Audio Result: Unhealthy (Confidence: 1.00)
ECG Result: Normal Person ECG Images (284x12=3408) (Confidence: 0.91)
PPG Result: Myocardial Infarction (Confidence: 1.00)
Final Risk Score: 0.63


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-20-4361906b9905>:124: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-20-4361906b9905>:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.6284172069857734

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models (Suppressing Compilation Warning)
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/ppg_scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder Safely
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixing 'Label' KeyError)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove non-numeric column
    else:
        print("Warning: 'Label' column not found in PPG data. Proceeding without it.")
    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # Load PPG Data Safely
    try:
        df_ppg = pd.read_excel(ppg_file)  # Read .xlsx instead of .csv
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})  # Convert 'MI' to 1 and 'Normal' to 0
        else:
            print("Warning: 'Label' column not found in PPG data.")
        ppg_row = df_ppg.dropna().iloc[0]  # Take the first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/UNHEALTHYSOUND.wav", "/content/MI(9).jpg", "/content/new_fixed_corrected.xlsx")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.6.1. Th

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Audio Result: Unhealthy (Confidence: 1.00)
ECG Result: ECG Images of Myocardial Infarction Patients (240x12=2880) (Confidence: 0.97)
PPG Result: Myocardial Infarction (Confidence: 1.00)
Final Risk Score: 0.89


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-2-699ea71b3854>:124: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-2-699ea71b3854>:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.8903404664824622

In [ ]:
!pip install streamlit opencv-python-headless joblib librosa tensorflow ngrok pandas xlrd


In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models (Suppressing Compilation Warning)
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder Safely
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixing 'Label' KeyError)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove non-numeric column
    else:
        print("Warning: 'Label' column not found in PPG data. Proceeding without it.")
    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # Load PPG Data Safely
    try:
        df_ppg = pd.read_excel(ppg_file)  # Read .xlsx instead of .csv
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})  # Convert 'MI' to 1 and 'Normal' to 0
        else:
            print("Warning: 'Label' column not found in PPG data.")
        ppg_row = df_ppg.dropna().iloc[0]  # Take the first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/a0011.wav", "/content/Normal(13).jpg", "/content/Normal.csv")


Error loading PPG data: Excel file format cannot be determined, you must specify an engine manually.


In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixed Feature Mismatch)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove target column

    # ✅ Ensure only 2000 features
    if len(data_row) > 2000:
        data_row = data_row.iloc[:2000]  # Trim extra features

    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # ✅ Load PPG Data
    try:
        df_ppg = pd.read_csv(ppg_file)

        # ✅ Fix 'Label' column if exists
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})

        ppg_row = df_ppg.dropna().iloc[0]  # Take first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/a0011.wav", "/content/Normal(13).jpg", "/content/Normal.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 630ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Audio Result: Healthy (Confidence: 1.00)
ECG Result: Normal Person ECG Images (284x12=3408) (Confidence: 0.87)
PPG Result: Normal (Confidence: 0.02)
Final Risk Score: 0.06


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-8-a28f2e9338df>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-8-a28f2e9338df>:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.0630960969362568

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixed Feature Mismatch)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove target column

    # ✅ Ensure only 2000 features
    if len(data_row) > 2000:
        data_row = data_row.iloc[:2000]  # Trim extra features

    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # ✅ Load PPG Data
    try:
        df_ppg = pd.read_csv(ppg_file)

        # ✅ Fix 'Label' column if exists
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})

        ppg_row = df_ppg.dropna().iloc[0]  # Take first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/a0011.wav", "/content/Normal(13).jpg", "/content/MI.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 631ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Audio Result: Healthy (Confidence: 1.00)
ECG Result: Normal Person ECG Images (284x12=3408) (Confidence: 0.87)
PPG Result: Myocardial Infarction (Confidence: 0.99)
Final Risk Score: 0.36


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-9-dd48b032544d>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-9-dd48b032544d>:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.36002896809439977

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixed Feature Mismatch)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove target column

    # ✅ Ensure only 2000 features
    if len(data_row) > 2000:
        data_row = data_row.iloc[:2000]  # Trim extra features

    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # ✅ Load PPG Data
    try:
        df_ppg = pd.read_csv(ppg_file)

        # ✅ Fix 'Label' column if exists
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})

        ppg_row = df_ppg.dropna().iloc[0]  # Take first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/e00670.wav", "/content/Normal(13).jpg", "/content/MI.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 841ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Audio Result: Unhealthy (Confidence: 1.00)
ECG Result: Normal Person ECG Images (284x12=3408) (Confidence: 0.87)
PPG Result: Myocardial Infarction (Confidence: 0.99)
Final Risk Score: 0.63


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-14-be0576f2a2e4>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-14-be0576f2a2e4>:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.6262791870421283

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixed Feature Mismatch)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove target column

    # ✅ Ensure only 2000 features
    if len(data_row) > 2000:
        data_row = data_row.iloc[:2000]  # Trim extra features

    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # ✅ Load PPG Data
    try:
        df_ppg = pd.read_csv(ppg_file)

        # ✅ Fix 'Label' column if exists
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})

        ppg_row = df_ppg.dropna().iloc[0]  # Take first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/a0016.wav", "/content/MI.jpg", "/content/Normal.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Audio Result: Healthy (Confidence: 1.00)
ECG Result: ECG Images of Patient that have abnormal heartbeat (233x12=2796) (Confidence: 1.00)
PPG Result: Normal (Confidence: 0.02)
Final Risk Score: 0.23


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-1-c54ff7dfb899>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-1-c54ff7dfb899>:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.23407972139456978

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixed Feature Mismatch)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove target column

    # ✅ Ensure only 2000 features
    if len(data_row) > 2000:
        data_row = data_row.iloc[:2000]  # Trim extra features

    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # ✅ Load PPG Data
    try:
        df_ppg = pd.read_csv(ppg_file)

        # ✅ Fix 'Label' column if exists
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})

        ppg_row = df_ppg.dropna().iloc[0]  # Take first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/a0016.wav", "/content/mi_2.jpg", "/content/Normal.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 693ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Audio Result: Healthy (Confidence: 1.00)
ECG Result: ECG Images of Patient that have abnormal heartbeat (233x12=2796) (Confidence: 0.52)
PPG Result: Normal (Confidence: 0.02)
Final Risk Score: 0.14


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-2-b80a1d0c7bb2>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-2-b80a1d0c7bb2>:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.13817559761145823

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixed Feature Mismatch)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove target column

    # ✅ Ensure only 2000 features
    if len(data_row) > 2000:
        data_row = data_row.iloc[:2000]  # Trim extra features

    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # ✅ Load PPG Data
    try:
        df_ppg = pd.read_csv(ppg_file)

        # ✅ Fix 'Label' column if exists
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})

        ppg_row = df_ppg.dropna().iloc[0]  # Take first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/a0016.wav", "/content/infmi_2x.jpg", "/content/Normal.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 676ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Audio Result: Healthy (Confidence: 1.00)
ECG Result: ECG Images of Patient that have abnormal heartbeat (233x12=2796) (Confidence: 1.00)
PPG Result: Normal (Confidence: 0.02)
Final Risk Score: 0.23


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-3-8a2a8c2b5ea6>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-3-8a2a8c2b5ea6>:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.23407972139456978

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import cv2
import pickle
import joblib
import pandas as pd
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ✅ Load Models
AUDIO_MODEL_PATH = "/content/SOUND_LSTM_model.h5"
ECG_MODEL_PATH = "/content/Resnet+EffieciencyNet+MLP_ecg_model (2).h5"
ENCODER_PATH = "/content/label_encoder.pkl"
PPG_MODEL_PATH = "/content/best_ppg_model.pkl"
SCALER_PATH = "/content/scaler.pkl"

audio_model = load_model(AUDIO_MODEL_PATH, compile=False)
ecg_model = load_model(ECG_MODEL_PATH, compile=False)
ppg_model = joblib.load(PPG_MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# ✅ Load Label Encoder
try:
    with open(ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
except Exception as e:
    print("Error loading LabelEncoder:", e)
    label_encoder = None

# ✅ Load Feature Extractors
base_model1 = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
base_model2 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, pooling="avg")

# ✅ Risk Weights
ECG_RISK_WEIGHTS = {
    "ECG Images of Myocardial Infarction Patients (240x12=2880)": 0.9,
    "ECG Images of Patient that have History of MI (172x12=2064)": 0.7,
    "ECG Images of Patient that have abnormal heartbeat (233x12=2796)": 0.6,
    "Normal Person ECG Images (284x12=3408)": 0.1
}
AUDIO_RISK_WEIGHTS = {"Healthy": 0.1, "Unhealthy": 0.9}
PPG_RISK_WEIGHTS = {0: 0.1, 1: 0.9}  # 0 = Normal, 1 = Myocardial Infarction

# ✅ Audio Preprocessing
def preprocess_audio(file_path, max_pad_len=400):
    audio, sample_rate = librosa.load(file_path, sr=2000)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    return mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

# ✅ ECG Preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# ✅ PPG Preprocessing (Fixed Feature Mismatch)
def preprocess_ppg(data_row):
    if "Label" in data_row:
        data_row = data_row.drop(columns=["Label"])  # Remove target column

    # ✅ Ensure only 2000 features
    if len(data_row) > 2000:
        data_row = data_row.iloc[:2000]  # Trim extra features

    data_row_scaled = scaler.transform([data_row])
    return data_row_scaled

# ✅ Predictions
def detect_heart_disease(audio_path):
    features = preprocess_audio(audio_path)
    prediction = audio_model.predict(features)[0]
    predicted_class = "Unhealthy" if prediction > 0.5 else "Healthy"
    confidence = prediction if predicted_class == "Unhealthy" else 1 - prediction
    return predicted_class, confidence

def predict_disease(ecg_path):
    img = preprocess_image(ecg_path)
    eff_features = base_model1.predict(img)
    res_features = base_model2.predict(img)
    final_features = np.hstack((eff_features, res_features))
    prediction = ecg_model.predict(final_features)
    predicted_class_index = np.argmax(prediction)

    if label_encoder:
        predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    else:
        predicted_class = f"Class_{predicted_class_index}"  # Fallback label

    return predicted_class, float(np.max(prediction))

def predict_ppg(ppg_row):
    processed_ppg = preprocess_ppg(ppg_row)
    prediction_prob = ppg_model.predict_proba(processed_ppg)[:, 1][0]
    predicted_class = 1 if prediction_prob > 0.5 else 0
    return predicted_class, prediction_prob

# ✅ Risk Calculation
def calculate_risk(audio_result, audio_confidence, ecg_result, ecg_confidence, ppg_result, ppg_confidence):
    audio_risk = AUDIO_RISK_WEIGHTS[audio_result] * audio_confidence
    ecg_risk = ECG_RISK_WEIGHTS.get(ecg_result, 0) * ecg_confidence
    ppg_risk = PPG_RISK_WEIGHTS[ppg_result] * ppg_confidence
    final_risk_score = (audio_risk + ecg_risk + ppg_risk) / 3
    return final_risk_score

def evaluate_patient(audio_path, ecg_path, ppg_file):
    # ✅ Load PPG Data
    try:
        df_ppg = pd.read_csv(ppg_file)

        # ✅ Fix 'Label' column if exists
        if "Label" in df_ppg.columns:
            df_ppg["Label"] = df_ppg["Label"].map({"Normal": 0, "MI": 1})

        ppg_row = df_ppg.dropna().iloc[0]  # Take first valid row
    except Exception as e:
        print("Error loading PPG data:", e)
        return None

    audio_result, audio_confidence = detect_heart_disease(audio_path)
    ecg_result, ecg_confidence = predict_disease(ecg_path)
    ppg_result, ppg_confidence = predict_ppg(ppg_row)
    risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))

    # ✅ Convert NumPy values to float for proper formatting
    print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")
    print(f"ECG Result: {ecg_result} (Confidence: {float(ecg_confidence):.2f})")
    print(f"PPG Result: {'Myocardial Infarction' if ppg_result == 1 else 'Normal'} (Confidence: {float(ppg_confidence):.2f})")
    print(f"Final Risk Score: {risk_score:.2f}")

    return risk_score

# ✅ Example Usage
evaluate_patient("/content/e00603.wav", "/content/MI(240).jpg", "/content/MI.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 807ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Audio Result: Unhealthy (Confidence: 1.00)
ECG Result: ECG Images of Myocardial Infarction Patients (240x12=2880) (Confidence: 0.79)
PPG Result: Myocardial Infarction (Confidence: 0.99)
Final Risk Score: 0.83


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-7-fd153e2767a5>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  risk_score = calculate_risk(audio_result, float(audio_confidence), ecg_result, float(ecg_confidence), ppg_result, float(ppg_confidence))
<ipython-input-7-fd153e2767a5>:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Audio Result: {audio_result} (Confidence: {float(audio_confidence):.2f})")


0.83416817966959